In [2]:
from utils.mf_dimensionality_reduction import *

train_numeric.csv loaded...
train_date.csv loaded...
test_numeric.csv loaded...
test_date.csv loaded...


# __<font color='blue'>Bosch Manufacturing</font>__

# __<font color='darkblue'> Part 4: Dimensionality Reduction</font>__

__Author:__ [Ryan Harper](www.kimrharper.com) <br><br>
__Data Source:__ [Bosch Dataset via Kaggle](https://www.kaggle.com/c/bosch-production-line-performance/data) <br> <br>
__Background:__ Bosch is a home appliance and industrial tools manufacturing company. In 2017, Bosch supplied Kaggle.com with manufacturing data to promote a competition. The goal of the competition was to determine factors that influence whether or not the product passes the final response stage of manufacturing and to predict which products are likely to fail based on this manufacturing process.<br> <br>
__The Data:__ Early exploration of this data will use a subset of the big data provided by Bosch. The data is provided by [Hitesh, John, and Matthew via PDX Data Science Meetup](https://www.meetup.com/Portland-Data-Science-Group/events/257370691/). The data subset is divided into 2 groups of 3 files (3 training, 3 test). Each group has one csv file each for numerical features ('numeric'), dates ('date'), and the manufacturing path ('cat'). The data subset includes a larger percentage of products that failed the response test, but not much more is known about this subsampling method.<br><br>
__Assumptions:__ ID # represents a specific product and that there is only one product. The differences in assembly are due to customization and/or differences between lines.<br><br>
__Goal:__ Predict which products will fail the response test. <br><br>

----

# 1. Dimensionality Reduction

### A. Import Data

In [4]:
# Retrieve vars from mf_stats_analysis.ipynb | quick var import
%store -r skewed_features
%store -r sig_diff_list
sig_diff_list.append(len(mf_num_data.columns)-1) # Adding response column index

In [21]:
ms_mcc = make_scorer(matthews_corrcoef)
TEST_PER = .3

def process_predictions(df):
    Xtr = df.iloc[:,:-1]
    Xtr = Xtr.fillna(Xtrdf.mean())
    Xtr = Xtr.dropna(axis=1, how='all')
    ytr = df.iloc[:,-1]
    Xte = mf_merged_test.fillna(mf_num_data_test.mean())
    Xte = Xte.dropna(axis=1, how='all')
    print(Xtr.values.shape)
    print(Xte.values.shape)
    print(ytr.values.shape)
        
    return Xtr.values, ytr.values, Xte.values

def save_to_csv(pred, title):
    final_predic = pd.DataFrame(mf_merged_test.iloc[:,0])
    print(len(final_pred))
    print(len(pred))
    final_predic['Response'] = pred
    pd.DataFrame(final_predic).to_csv(title) 
    
def final_predictions(data, title):
    X_train, y_train, X_test = process_predictions(data)
    rf = RandomForestClassifier(n_estimators=100, criterion='entropy', n_jobs=4, class_weight='balanced', verbose=1)
    print(y_train)
    rf.fit(X_train, y_train)
    predictions = rf.predict(X_test)
    save_to_csv(predictions, title)

In [7]:
transformed_df = merged_df
for f in skewed_features:
    transformed_df[f] = np.log(1+ transformed_df[f])
all_transformed_df = merged_df
for f in list(all_transformed_df.columns[1:-1]):
    all_transformed_df[f] = np.log(1+ all_transformed_df[f])    

In [23]:
final_predictions(mf_merged_train, 'RH-ND-NoTransform.csv')

(139484, 2115)
(15498, 2114)
(139484,)
[1009.2530431 1009.2530431 1009.2530431 ... 1009.2530431 1009.2530431
 1009.2530431]


ValueError: Unknown label type: 'continuous'

In [ ]:
final_predictions(transformed_df, 'RH-ND-SkewTransform.csv')

In [ ]:
final_predictions(all_transformed_df, 'RH-ND-AllTransform.csv')

In [ ]:
transformed_df = merged_df
for f in skewed_features:
    transformed_df[f] = np.log(1+ transformed_df[f])
X_train, X_test, y_train, y_test = process_data(transformed_df, True)
rf = RandomForestClassifier(n_estimators=100, criterion='entropy', n_jobs=4,class_weight='balanced', verbose=1)
rf.fit(X_train, y_train)
y_predict = rf.predict(X_test)
print(matthews_corrcoef(y_test, y_predict))

In [ ]:
all_transformed_df = merged_df
for f in list(all_transformed_df.columns[1:-1]):
    all_transformed_df[f] = np.log(1+ all_transformed_df[f])
X_train, X_test, y_train, y_test = process_data(all_transformed_df, True)
rf = RandomForestClassifier(n_estimators=100, criterion='entropy', n_jobs=4,class_weight='balanced', verbose=1)
rf.fit(X_train, y_train)
y_predict = rf.predict(X_test)
print(matthews_corrcoef(y_test, y_predict))

### B. Functions and Declarations

In [ ]:
mf_num_data.head()

### C. Dimensionality Reduction

#### __PCA__

In [ ]:
column_names = ['param_model','param_reduce_dim__n_components','param_model__max_depth','param_model__n_estimators','mean_test_score','rank_test_score','mean_fit_time']
display_results(grid, columns, 'visuals/pca_models_fixed_imbalance.html')

# 2. Feature Selection

#### __RFE__

__Recreate Train/Test Data:__

In [ ]:
X_train, X_test, y_train, y_test = process_data(mf_num_data.iloc[:,sig_diff_list], False)

In [ ]:
%%time
estimator = RandomForestClassifier()
selector = RFE(estimator, step=.05)
selector = selector.fit(X_train, y_train)
RFE_selection = pd.DataFrame(list(zip(mf_num_data.columns[1:-1],selector.ranking_)))

In [ ]:
top_features = RFE_selection[0][RFE_selection[1] <= 1].values
X, features = process_data(mf_num_data[top_features], True)
y = mf_num_data.iloc[:,-1]
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=test_per, random_state=42)

__RF Run 1:__

In [ ]:
rf = RandomForestClassifier(n_estimators=200, class_weight='balanced')
rf.fit(X_train, y_train)
y_predict = rf.predict(X_test)
print(matthews_corrcoef(y_test, y_predict))

__RF Run 2:__

In [ ]:
rf = RandomForestClassifier(n_estimators=100, criterion='entropy', n_jobs=4,class_weight='balanced', verbose=1)
rf.fit(X_train, y_train)
y_predict = rf.predict(X_test)
print(matthews_corrcoef(y_test, y_predict))

__RF Run 3:__

In [ ]:
rf = RandomForestClassifier(n_estimators=100, criterion='entropy', n_jobs=4,class_weight='balanced', verbose=1)
rf.fit(X_train, y_train)
y_predict = rf.predict(X_test)
print(matthews_corrcoef(y_test, y_predict))

# 3. Feature Addition

In [ ]:
merged_df = mf_num_data.append(mf_date_data)
# merged_df.isna().any()[lambda x: x]
merged_df = merged_df.fillna(merged_df.mean())
drop_nans = list(merged_df.isna().any()[lambda x: x].index)
merged_df.drop(drop_nans, axis=1, inplace=True)

In [ ]:
X_train, X_test, y_train, y_test = process_data(merged_df, True)

In [ ]:
estimator = RandomForestClassifier(criterion='entropy', n_jobs=4,class_weight='balanced')
selector = RFE(estimator, n_features_to_select=100, step=20)
selector = selector.fit(X_train, y_train)
RFE_selection = pd.DataFrame(list(zip(mf_num_data.columns[1:-1],selector.ranking_)))

In [ ]:
get_probs = []
for i in range(1,20):
    top_features = RFE_selection[0][RFE_selection[1] == i].values
    top_features2 = list(top_features)
    top_features2.append('Response')
    X_train, X_test, y_train, y_test = process_data(merged_df[top_features2], True)
    rf = RandomForestClassifier()
    rf.fit(X_train, y_train)
    y_predict = rf.predict(X_test)
    print(matthews_corrcoef(y_test, y_predict))
    get_probs.append(rf.predict_proba(X_test))

In [ ]:
newDF = pd.DataFrame()
for r in range(len(get_probs)):
    r_prob = get_probs[r]
    newDF[r] = r_prob[:,0]
    success_prob = newDF.mean(axis=1)
    
    r_prob = get_probs[r]
    newDF[r] = r_prob[:,1]
    fail_prob = newDF.mean(axis=1)

make_predictions = pd.DataFrame([success_prob, fail_prob]).T

In [ ]:
def compare(a,b):
    return 0 if a > b else 1

In [ ]:
results = [compare(make_predictions.iloc[i,:][0],make_predictions.iloc[i,:][1]) for i in range(len(make_predictions))]

In [ ]:
print(matthews_corrcoef(y_test, results))

------

__Best Prediction:__ <br>
0.41037390395473183<br>RandomForestClassifier(n_estimators=100, criterion='entropy', n_jobs=4,class_weight='balanced', verbose=1) <br>RFE(estimator, step=.05)